In [91]:
from pyscf import gto,scf
import numpy as np
import matplotlib.pyplot as plt
from FcMole import *
from AP_utils import DeltaV
from numpy.linalg import norm
from alch_deriv import alchemy_cphf_deriv,make_dP

In [265]:
CO=gto.M(atom="C 0 0 0; O 0 0 2 ",unit="Bohr", basis="def2-qzvp")

In [266]:
mf=scf.RKS(CO)
mf.xc="PBE0"
mf.scf()
P0=mf.make_rdm1()
C0=mf.mo_coeff

converged SCF energy = -113.225870191075


In [328]:
CO1=FcM_like(CO,fcs=[.5,0])
mf1=scf.RKS(CO1)
mf1.xc="PBE0"
mf1.scf(dm0=P0)
P1=mf1.make_rdm1()
C1=mf1.mo_coeff

converged SCF energy = -120.958557196315


In [268]:
dV=DeltaV(CO,[[0],[.1]])

In [269]:
Delt_e_elec=mf1.energy_elec()[0]-mf.energy_elec()[0]
np.einsum("ij,ij",P0,dV),Delt_e_elec  # APDFT1 predictions 

(-1.8714772182529278, -20.229884894502817)

In [270]:
from pyscf.scf import cphf
from pyscf.scf import  _response_functions

In [271]:
rf=mf.gen_response(mf.mo_coeff, mf.mo_occ, hermi=1)
dm1test=np.ones((10,10))/100
dm1test-=np.sum(dm1test*P0)/norm(P0)**2*P0
respfun=rf(dm1test)
dFt=mf.get_fock(dm=P0+dm1test)-mf.get_fock(dm=P0)
np.allclose(dFt,respfun),norm(dFt-respfun),norm(dFt)

ValueError: operands could not be broadcast together with shapes (10,10) (114,114) 

In [ ]:
from pyscf.hessian.rhf import gen_vind
from pyscf.scf import cphf
from pyscf import lib

mo_energy,mo_coeff, mo_occ=mf.mo_energy,mf.mo_coeff, mf.mo_occ
vind = gen_vind(mf, mf.mo_coeff, mf.mo_occ)

In [272]:
mol=CO
int_r=DeltaV(CO,[[0],[.01]])
occidx = mo_occ > 0
orbo = mo_coeff[:, occidx]
orbv = mo_coeff[:,~occidx]
charges = mol.atom_charges()
coords  = mol.atom_coords()
charge_center = np.einsum('i,ix->x', charges, coords) / charges.sum()
h1 = lib.einsum('pq,pi,qj->ij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals
#h1=h1.reshape((1,h1.shape[0],h1.shape[1]))

ValueError: Size of label 'p' for operand 1 (114) does not match previous terms (10).

In [273]:
h1.shape

(10, 7)

In [274]:
#mol = mf.mol
#mo_energy = mf.mo_energy
#mo_coeff = mf.mo_coeff
#mo_occ = mf.mo_occ
#occidx = mo_occ > 0
#orbo = mo_coeff[:, occidx]
#orbv = mo_coeff[:,~occidx]
#charges = mol.atom_charges()
#coords  = mol.atom_coords()
#charge_center = np.einsum('i,ix->x', charges, coords) / charges.sum()
#h1 = lib.einsum('pq,pi,qj->ij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals
#h1=h1.reshape((1,h1.shape[0],h1.shape[1]))
#s1 = np.zeros_like(h1)
#vind = gen_vind(mf, mo_coeff, mo_occ)
#mo1,e1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1,40, 1e-9)
#        s1=None
#mo1,e1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1,10,1e-6)

In [275]:
mo1,e1=alchemy_cphf_deriv(mf,int_r)

In [276]:
dPfd=P1-P0

In [277]:
dP_ad=make_dP(mf,mo1)

In [278]:
norm(dPfd),norm(dPfd-dP_ad)

(1.468601302144129, 1.4483446235338027)

In [334]:
from AP_class import APDFT_perturbator as AP

In [335]:
ap=AP(mf,sites=[0])
ap.build_all()

In [336]:
ap.APDFT1([.5]),mf1.e_tot

(-120.58325628234012, -120.95855720373132)

In [337]:
ap.APDFT2([.5]),mf1.e_tot

(-120.95525995031093, -120.95855720373132)

In [338]:
ap.APDFT3([.5]),mf1.e_tot

(-120.95909509119817, -120.95855720373132)

In [339]:
ap.af(0)/2

libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WA

array([[-1.73491072e-15,  5.44887860e-15, -1.40899059e-01],
       [ 1.79970257e-15, -5.03822111e-15,  1.22447291e-01]])

In [340]:
from pyscf.grad import rks


In [320]:
mf1.Gradients().grad()-mf.Gradients().grad()

libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- RKS gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0798953447
1 O    -0.0000000000     0.0000000000    -0.0798993333
----------------------------------------------


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.1968481851
1 O     0.0000000000     0.0000000000    -0.1968551477
----------------------------------------------


array([[ 6.96155082e-15, -1.48347324e-15, -1.16952840e-01],
       [-7.00158508e-15,  2.78014606e-15,  1.16955814e-01]])

In [322]:
from pyscf.geomopt import optimize

In [332]:
optimize(mf1,trust=.01)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   O   0.000000   0.000000   1.058354    0.000000  0.000000  0.000000
converged SCF energy = -120.958557196612


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.0799016684
1 O    -0.0000000000     0.0000000000   -28.5380588582
----------------------------------------------
cycle 1: E = -120.958557197  dE = -120.959  norm(grad) = 28.5382

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000  -0.002646    0.000000  0.000000 -0.002646
   O   0.000000   0.000000   1.061000    0.000000  0.000000  0.002646
converged SCF energy = -120.959263345249


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    21.2084826242
1 O     0.0000000000     0.0000000000    -2.2437094558
----------------------------------------------
cycle 2: E = -120.959263345  dE = -0.000706149  norm(grad) = 21.3268

Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000  -0.001268    0.000000  0.000000  0.001378
   O   0.000000   0.000000   1.059622    0.000000  0.000000 -0.001378
converged SCF energy = -120.958918505634


...

libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000    17.2820642147
1 O     0.0000000000    -0.0000000000    -7.1320266985
----------------------------------------------
cycle 6: E = -120.959140968  dE = -3.84596e-05  norm(grad) = 18.6959

Geometry optimization cycle 7
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000  -0.002244    0.000000  0.000000 -0.000111
   O   0.000000   0.000000   1.060599    0.000000  0.000000  0.000111
converged SCF energy = -120.95916800488


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000    18.1379461863
1 O    -0.0000000000     0.0000000000    -6.0665566744
----------------------------------------------
cycle 7: E = -120.959168005  dE = -2.70366e-05  norm(grad) = 19.1256


RuntimeError: The trust radius got too small, check forces?

In [333]:
mf1.scf(dm0=P1)

converged SCF energy = -120.958557203731


-120.95855720373132